In [13]:
import sys; sys.path.append("..")

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# config is located in the repo
from config import *

# settings
Since my dataset contains 1-month USDRUB derivatives, I assume the maturity of 30 days; I use the USD OIS rates as the USD interest rate. Together, this implies an annualization factor of $360/30$.

In [15]:
DATA_PATH = Path(os.environ.get("CONTENT_ROOT", ".")) / "data"

filename_raw_data = "usdrub-data-hf.ftr"
rates_ann_factor = 360 / 30
EXAMPLE_DATE = "2022-02-24 03:48:00+01:00"

# raw data
The raw data is a collection of USDRUB spot and forward prices and FX option contracts plus the USD OIS rate, all stacked in the long-form as a .ftr pyarrow file with columns ['date', 'contract', 'value']. The (contract, value) pairs are:
| contract  | value
| --------- | -----
| spot      | USDRUB spot rate
| forward   | USDRUB 1-month forward rate
| v_atm     | volatility of 1-month ATM contracts, in percent.
| v_25r     | quote of 25-delta risk reversal, in percent
| v_10r     | quote of 10-delta risk reversal, in percent
| v_25b     | quote of 25-delta butterfly spread (market strangle), in percent
| v_10b     | quote of 10-delta butterfly spread (market strangle), in percent
| r_base    | risk-free rate in the base currency, in percent.

In [16]:
data = pd.read_feather(DATA_PATH / "raw" / filename_raw_data)

print(f"number of unique dates: {len(data['date'].unique())}")
print(f"sample on {EXAMPLE_DATE[:16]}")
data.query(f"date == '{EXAMPLE_DATE}'").reset_index(drop=True)\
    .style.format(precision=2)

number of unique dates: 57307
sample on 2022-02-24 03:48


,date,contract,value
0,2022-02-24 03:48:00+01:00,spot,81.24
1,2022-02-24 03:48:00+01:00,r_base,0.23
2,2022-02-24 03:48:00+01:00,v_atm,35.83
3,2022-02-24 03:48:00+01:00,v_25r,12.21
4,2022-02-24 03:48:00+01:00,v_10r,26.61
5,2022-02-24 03:48:00+01:00,v_25b,1.50
6,2022-02-24 03:48:00+01:00,v_10b,7.47
7,2022-02-24 03:48:00+01:00,forward,82.16


In [17]:
# pivot for easier access
data = data.pivot(index="date", columns="contract", values="value")

It is pretty safe to forward-fill the USD interest rate, as it barely changes at this frequency.

In [18]:
data["r_base"] = data["r_base"].ffill()

Also, we have to drop rows with at least one missing value, as all values are essential to our analysis.

In [19]:
# data = data.dropna(how="any")

We need to:
- introduce one new variable: the interest rate for the counter currency; 
- de-annualize all relevant values (interest rates and the option contract quotes expressed as volatilities);
- express the latter in fractions of 1 instead of percent; and finally 
- convert interest rates to continuously compounded as required by the Black-Scholes formulas.

In [20]:
# from percent to fractions of 1 where relevant
data[['r_base', 'v_atm', 'v_25r', 'v_10r', 'v_25b', 'v_10b']] /= 100

# calculate interest rate for the counter currency
# f = s * (1 + r_counter) / (1 + r_base)
data["r_counter"] = (
        data["forward"] / data["spot"] *
            (1 + data["r_base"] / rates_ann_factor)
        - 1
) * rates_ann_factor

# rates to continuously compounded
data[["r_base", "r_counter"]] = np.log(1 + data[["r_base", "r_counter"]])

# unpivot
data = data.reset_index().melt(id_vars="date", value_name="value",
                               var_name="contract")

data.query(f"date == '{EXAMPLE_DATE}'").reset_index(drop=True)\
    .style.format(precision=4)

,date,contract,value
0,2022-02-24 03:48:00+01:00,forward,82.1610
1,2022-02-24 03:48:00+01:00,r_base,0.0023
2,2022-02-24 03:48:00+01:00,spot,81.2420
3,2022-02-24 03:48:00+01:00,v_10b,0.0747
4,2022-02-24 03:48:00+01:00,v_10r,0.2661
5,2022-02-24 03:48:00+01:00,v_25b,0.0150
6,2022-02-24 03:48:00+01:00,v_25r,0.1221
7,2022-02-24 03:48:00+01:00,v_atm,0.3583
8,2022-02-24 03:48:00+01:00,r_counter,0.1293


In [21]:
# store in data/processed/
data.to_feather(DATA_PATH / "processed" / "data.ftr")